# Blink Problem 
Please refer to the material of week 1 of course for more details about this problem.
Blink has 2 facilities that supplies materials to three different regions.

## Problem Objective
How much sand should SandyCo ship from each plant to each region
each week to meet demand at the lowest planned distribution costs?

## Input Data


In [1]:

D = [2500, 4350, 3296]    # quantity demanded at region
distance = [
        [105, 256, 86],
        [240, 136, 198]
    ]           # costs from i to j
c = 0.12

In [2]:

# defining number of plants and customer's region
num_plants = len(distance)
num_regions = len(D)

In [3]:
from ortools.linear_solver import pywraplp

In [4]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')


# Declare an Matrix to hold our variables for each x_ij.
# When defining the range of the variable, I have already taken into account the constraint x>= 0.
x = {}
for i in range(num_plants):
    for j in range(num_regions):
        x[i,j] = solver.NumVar(0, solver.infinity(), "")

print('Number of variables =', solver.NumVariables())

# Demand requirement
## ex. x11 + x21 >= D1
for j in range(num_regions):
    solver.Add(solver.Sum([x[i,j] for i in range(num_plants)]) >= D[j])

print('Number of constraints =', solver.NumConstraints())

## sum of the cost per ton times the total volume
## min [c11*x11 + c12*c12 + ... ]
solver.Minimize(solver.Sum([c*distance[i][j] * x[i,j] for i in range(num_plants) for j in range(num_regions)]))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()} per month \n ')
    for i in range(num_plants):
        for j in range(num_regions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            print(f'Plant {i} supplies ' +
                  f'{x[i,j].solution_value()} Units '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())




Number of variables = 6
Number of constraints = 3
Total cost = 136506.72 per month 
 
Plant 0 supplies 2500.0 Units to Region 0
Plant 0 supplies 0.0 Units to Region 1
Plant 0 supplies 3296.0 Units to Region 2
Plant 1 supplies 0.0 Units to Region 0
Plant 1 supplies 4350.0 Units to Region 1
Plant 1 supplies 0.0 Units to Region 2

Advanced usage:
Problem solved in 7.000000 milliseconds
Problem solved in 0 iterations


# part 2 

In [5]:
p = 45
# Create the mip solverII with the SCIP backend.
solverII = pywraplp.Solver.CreateSolver('SCIP')


# Declare an Matrix to hold our variables for each x_ij.
# When defining the range of the variable, I have already taken into account the constraint x>= 0.
x = {}
for i in range(num_plants):
    for j in range(num_regions):
        x[i,j] = solverII.NumVar(0, solverII.infinity(), "")

print('Number of variables =', solverII.NumVariables())

# Demand requirement
## now making the constraint equals to the demand
for j in range(num_regions):
    solverII.Add(solverII.Sum([x[i,j] for i in range(num_plants)]) == D[j])

print('Number of constraints =', solverII.NumConstraints())

Number of variables = 6
Number of constraints = 3


In [6]:
transportation_cost = solverII.Sum([c * distance[i][j] * x[i,j] for i in range(num_plants) for j in range(num_regions)])
revenue = solverII.Sum([p * x[i,j] for i in range(num_plants) for j in range(num_regions)])
solverII.Maximize(revenue - transportation_cost)

In [7]:
statusII = solverII.Solve()

In [8]:
pywraplp.Solver.FEASIBLE

1

In [9]:
#print the mathematical model 
print(solverII.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 3
   Variables        : 6
     Binary         : 0
     Integer        : 0
     Continuous     : 6
Maximize
 Obj: +32.4 auto_v_000000000 +14.28 auto_v_000000001 +34.68 auto_v_000000002 +16.2 auto_v_000000003 +28.68 auto_v_000000004 +21.24 auto_v_000000005 
Subject to
 auto_c_000000000: +1 auto_v_000000000 +1 auto_v_000000003  = 2500
 auto_c_000000001: +1 auto_v_000000001 +1 auto_v_000000004  = 4350
 auto_c_000000002: +1 auto_v_000000002 +1 auto_v_000000005  = 3296
Bounds
 0 <= auto_v_000000000
 0 <= auto_v_000000001
 0 <= auto_v_000000002
 0 <= auto_v_000000003
 0 <= auto_v_000000004
 0 <= auto_v_000000005
End



In [10]:
if statusII == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solverII.Objective().Value()} per month \n ')
    for i in range(num_plants):
        for j in range(num_regions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            print(f'Plant {i} supplies ' +
                  f'{x[i,j].solution_value()} Units '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solverII.wall_time())
print('Problem solved in %d iterations' % solverII.iterations())

Total cost = 320063.27999999997 per month 
 
Plant 0 supplies 2500.0 Units to Region 0
Plant 0 supplies 0.0 Units to Region 1
Plant 0 supplies 3296.0 Units to Region 2
Plant 1 supplies 0.0 Units to Region 0
Plant 1 supplies 4350.0 Units to Region 1
Plant 1 supplies 0.0 Units to Region 2

Advanced usage:
Problem solved in 105.000000 milliseconds
Problem solved in 0 iterations


## part 3

In [11]:
S = [13000, 3500 ]

In [12]:
# Create the mip solverIII with the SCIP backend.
solverIII = pywraplp.Solver.CreateSolver('SCIP')


# Declare an Matrix to hold our variables for each x_ij.
# When defining the range of the variable, I have already taken into account the constraint x>= 0.
x = {}
for i in range(num_plants):
    for j in range(num_regions):
        x[i,j] = solverIII.NumVar(0, solverIII.infinity(), "")

print('Number of variables =', solverIII.NumVariables())

## new constraint of Supply limit 
for i in range(num_plants):
    solverIII.Add(solverIII.Sum([x[i,k] for k in range(num_plants)]) <= S[i])
    
# Demand requirement
## ex. x11 + x21 >= D1
for j in range(num_regions):
    solverIII.Add(solverIII.Sum([x[i,j] for i in range(num_plants)]) >= D[j])

print('Number of constraints =', solverIII.NumConstraints())

## sum of the cost per ton times the total volume
## min [c11*x11 + c12*c12 + ... ]
solverIII.Minimize(solverIII.Sum([c*distance[i][j] * x[i,j] for i in range(num_plants) for j in range(num_regions)]))

statusIII = solverIII.Solve()

if statusIII == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solverIII.Objective().Value()} per month \n ')
    for i in range(num_plants):
        for j in range(num_regions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            print(f'Plant {i} supplies ' +
                  f'{x[i,j].solution_value()} Units '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solverIII.wall_time())
print('Problem solved in %d iterations' % solverIII.iterations())




Number of variables = 6
Number of constraints = 5
Total cost = 148746.72 per month 
 
Plant 0 supplies 2500.0 Units to Region 0
Plant 0 supplies 850.0 Units to Region 1
Plant 0 supplies 3296.0 Units to Region 2
Plant 1 supplies 0.0 Units to Region 0
Plant 1 supplies 3500.0 Units to Region 1
Plant 1 supplies 0.0 Units to Region 2

Advanced usage:
Problem solved in 8.000000 milliseconds
Problem solved in 0 iterations


# Part4

In [13]:
c_inbound = .04
c_outbound = 0.08

distance_inbound = [[190] , [150]]
distance_outbound = [15, 59, 79]
num_dc = len(distance_inbound[0])

len(distance_inbound)

# Create the mip solverIV with the SCIP backend.
solverIV = pywraplp.Solver.CreateSolver('SCIP')


# Declare an Matrix to hold our variables for each x_ij.
# When defining the range of the variable, I have already taken into account the constraint x>= 0.
x = {}
for i in range(num_plants):
    for k in range(num_dc):
        x[i,k] = solverIV.NumVar(0, solverIV.infinity(), "")
y = {}
for k in range(num_dc):
    for j in range(num_regions):
        y[k,j] = solverIV.NumVar(0, solverIV.infinity(), "")

print('Number of variables =', solverIV.NumVariables())

## new constraint of Supply limit 
for i in range(num_plants):
    for k in range(num_dc):
        solverIV.Add( x[i, k] <= S[i])
    
# Demand requirement
for k in range(num_dc):
    for j in range(num_regions):
        solverIV.Add(y[k, j] >= D[j])
    
inbound_flow = solverIV.Sum([x[i, k] for i in range(num_plants) for k in range(num_dc)])
outbound_flow = solverIV.Sum([y[k, j] for k in range(num_dc) for j in range(num_regions)])
solverIV.Add(inbound_flow == outbound_flow)

    
    
print('Number of constraints =', solverIV.NumConstraints())

## sum of the cost per ton times the total volume
cost_inbound = solverIV.Sum([c_inbound * distance_inbound[i][k] * x[i, k] for i in range(num_plants) for k in range(num_dc)])
cost_outbound = solverIV.Sum([c_outbound*distance_outbound[j] * y[k,j] for k in range(num_dc) for j in range(num_regions)])
solverIV.Minimize(cost_inbound + cost_outbound )

statusIV = solverIV.Solve()

if statusIV == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solverIV.Objective().Value()} per month \n')
    for i in range(num_plants):
        for k in range(num_dc):
            print(f'Plant {i} sends ' +
                  f'{x[i,k].solution_value()} tons '+
                 f'to Center {k}')
            
    for k in range(num_dc):
        for j in range(num_regions):
            print(f'CD {k} supplies ' +
                  f'{y[k,j].solution_value()} tons '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solverII.wall_time())
print('Problem solved in %d iterations' % solverII.iterations())

Number of variables = 5
Number of constraints = 6
Total cost = 115872.32 per month 

Plant 0 sends 6646.0 tons to Center 0
Plant 1 sends 3500.0 tons to Center 0
CD 0 supplies 2500.0 tons to Region 0
CD 0 supplies 4350.0 tons to Region 1
CD 0 supplies 3296.0 tons to Region 2

Advanced usage:
Problem solved in 179.000000 milliseconds
Problem solved in 0 iterations


## Part 5

In [40]:
fixed_cost = [11500, 15500]

distance_inbound = [[190, 15] , [150, 36]]
distance_outbound = [[15, 59, 79],
                    [135, 45, 129]]
num_dc = len(distance_inbound[0])

len(distance_inbound)

# Create the mip solverV with the SCIP backend.
solverV = pywraplp.Solver.CreateSolver('SCIP')


# Declare an Matrix to hold our variables for each x_ij.
# When defining the range of the variable, I have already taken into account the constraint x>= 0.
x = {}
for i in range(num_plants):
    for k in range(num_dc):
        x[i,k] = solverV.NumVar(0, solverV.infinity(), "")
y = {}
for k in range(num_dc):
    for j in range(num_regions):
        y[k,j] = solverV.NumVar(0, solverV.infinity(), "")

z={}
for k in range(num_dc):
    z[k] = solverV.BoolVar("")

print('Number of variables =', solverV.NumVariables())

Number of variables = 12


In [41]:
M = sum(D) # M is a very big number in this case equals to the total demand

In [42]:
## new constraint of Supply limit 
for i in range(num_plants):
    solverV.Add(solverV.Sum([x[i, k] for k in range(num_dc)])<= S[i])
    
# Demand requirement

for j in range(num_regions):
    solverV.Add(solverV.Sum([y[k, j] for k in range(num_dc)])>= D[j])
        
# conservation of flow
for k in range(num_dc):
    inbound_flow = solverV.Sum([x[i, k] for i in range(num_plants)])
    outbound_flow = solverV.Sum([y[k, j] for j in range(num_regions)])
    solverV.Add(inbound_flow == outbound_flow)

# #max number opf cds open
# solverV.Add(solverV.Sum([z[k] for k in range(num_dc)]) == 1)

#linking constraint
for k in range(num_dc):
    solverV.Add(solverV.Sum([x[i,k] for i in range(num_plants)])  <= 999999 *z[k])


print('Number of constraints =', solverV.NumConstraints())

Number of constraints = 9


In [43]:
## sum of the cost per ton times the total volume
cost_inbound = solverV.Sum([c_inbound * distance_inbound[i][k] * x[i, k] for i in range(num_plants) for k in range(num_dc)])
cost_outbound = solverV.Sum([c_outbound*distance_outbound[k][j] * y[k,j] for k in range(num_dc) for j in range(num_regions)])
fixed_cost_O = solverV.Sum([fixed_cost[k] * z[k] for k in range(num_dc) ])
solverV.Minimize(cost_inbound + cost_outbound + fixed_cost_O)

statusV = solverV.Solve()

if statusV == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solverV.Objective().Value()} per month \n')
    for i in range(num_plants):
        for k in range(num_dc):
            print(f'Plant {i} sends ' +
                  f'{x[i,k].solution_value()} tons '+
                 f'to Center {k}')
            
    for k in range(num_dc):
        for j in range(num_regions):
            print(f'CD {k} supplies ' +
                  f'{y[k,j].solution_value()} tons '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solverII.wall_time())
print('Problem solved in %d iterations' % solverII.iterations())

Total cost = 98262.31999999999 per month 

Plant 0 sends 0.0 tons to Center 0
Plant 0 sends 10146.0 tons to Center 1
Plant 1 sends 0.0 tons to Center 0
Plant 1 sends 0.0 tons to Center 1
CD 0 supplies 0.0 tons to Region 0
CD 0 supplies 0.0 tons to Region 1
CD 0 supplies 0.0 tons to Region 2
CD 1 supplies 2500.0000000000005 tons to Region 0
CD 1 supplies 4349.999999999999 tons to Region 1
CD 1 supplies 3295.999999999999 tons to Region 2

Advanced usage:
Problem solved in 430756.000000 milliseconds
Problem solved in 0 iterations


In [39]:
#print the mathematical model 
print(solverV.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 10
   Variables        : 12
     Binary         : 2
     Integer        : 0
     Continuous     : 10
Minimize
 Obj: +7.6 auto_v_000000000 +0.6 auto_v_000000001 +6 auto_v_000000002 +1.44 auto_v_000000003 +1.2 auto_v_000000004 +4.72 auto_v_000000005 +6.32 auto_v_000000006 +10.8 auto_v_000000007 +3.6 auto_v_000000008 +10.32 auto_v_000000009 +11500 auto_v_000000010 +15500 auto_v_000000011 
Subject to
 auto_c_000000000: +1 auto_v_000000000 +1 auto_v_000000001  <= 13000
 auto_c_000000001: +1 auto_v_000000002 +1 auto_v_000000003  <= 3500
 auto_c_000000002: +1 auto_v_000000004 +1 auto_v_000000007  >= 2500
 auto_c_000000003: +1 auto_v_000000005 +1 auto_v_000000008  >= 4350
 auto_c_000000004: +1 auto_v_000000006 +1 auto_v_000000009  >= 3296
 auto_c_000000005: +1 auto_v_000000000 +1 auto_v_000000002 -1 auto_v_000000004 -1 auto_v_000000005 -1 auto_v_000000006  = 0
 auto_c_000000006: +1 auto_v